In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from collections import defaultdict

import matplotlib

matplotlib.rcParams['font.family'] ='Malgun Gothic'

matplotlib.rcParams['axes.unicode_minus'] =False

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score, accuracy_score, precision_score
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import load_model
import h5py

In [11]:
import datetime

import pymysql
import MySQLdb

import FinanceDataReader as fdr

from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

import fbprophet
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

# 수익률정리

In [344]:
files = os.listdir('c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/3개월/train/')
name_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/kospi_목록_코드_종목코드.csv',index_col=0)
code_li = []
for code in name_df.stock_code:
    code_li.append(f'{code:06}')
name_df.stock_code = code_li

In [365]:
precision_li = []
roc_auc_li = []
f1_score_li = []
accuracy_li = []
name_li = []
stockcode_li = []

test_files = os.listdir('c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/3개월/test/')

fail_li = []

for test in test_files:
    
    test_df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/3개월/test/{test}',index_col=0)
    y_test = test_df['15per']
    x_test = test_df.dropna().drop(['date','등락률','수익률','종목명','quarter','5per','10per','15per'], axis=1)

    with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/3개월_15per/Randomforest/{test[:6]}.pkl','rb') as f:
        rfc = pickle.load(f)

    pred = rfc.predict(x_test)  
    prob = rfc.predict_proba(x_test)

    stock_name = str(name_df[name_df.stock_code == test[:6]]['corp_name'].values)
    name_li.append(stock_name)
    stockcode_li.append(test[:6])
    # print(stock_name, ' : ',test[:6])
    
    try:
        roc_auc_li.append(roc_auc_score(y_test, prob[:,1]))
        precision_li.append(precision_score(y_test, pred))
        f1_score_li.append(f1_score(y_test, pred))
        accuracy_li.append(accuracy_score(y_test, pred))
        
    except:  
        fail_li.append(test[:6])
        precision_li.append(precision_score(y_test, pred))
        roc_auc_li.append(np.nan)
        f1_score_li.append(f1_score(y_test, pred))
        accuracy_li.append(accuracy_score(y_test, pred))
    # print('-'*100)

In [366]:
len(precision_li)

491

In [367]:
rfc_3m = pd.DataFrame({'code':stockcode_li,
                        'name': name_li,
                        'precision': precision_li,
                        'f1_score': f1_score_li,
                        'accuracy': accuracy_li,
                        'roc_auc': roc_auc_li})
rfc_3m

,code,name,precision,f1_score,accuracy,roc_auc
0,000040,['KR모터스'],0.0,0.0,0.991935,NaN
1,000050,['경방'],0.0,0.0,0.991935,0.508130
2,000080,['하이트진로'],0.0,0.0,0.822581,0.899287
3,000100,['유한양행'],0.0,0.0,0.975806,NaN
4,000120,['CJ대한통운'],0.0,0.0,1.000000,NaN
...,...,...,...,...,...,...
486,214320,['이노션'],0.0,0.0,1.000000,NaN
487,214330,['금호에이치티'],0.0,0.0,0.967742,0.817073
488,214420,['토니모리'],0.0,0.0,0.346774,NaN
489,226320,['잇츠한불'],0.0,0.0,0.919355,0.922368


In [368]:
rfc_3m.to_csv('c:/Users/bitcamp/Desktop/final_data/모델정리/종목별_머신러닝/전처리전/rfc_3m_15.csv')

In [369]:
rfc_3m['precision'] = rfc_3m['precision'].apply(lambda x: np.nan if x == 1 else x)
rfc_3m['f1_score'] = rfc_3m['f1_score'].apply(lambda x: np.nan if x == 1 else x)
rfc_3m['accuracy'] = rfc_3m['accuracy'].apply(lambda x: np.nan if x == 1 else x)
rfc_3m['roc_auc'] = rfc_3m['roc_auc'].apply(lambda x: np.nan if x == 1 else x)

In [370]:
rfc_3m['precision'] = rfc_3m['precision'].apply(lambda x: np.nan if x == 0 else x)
rfc_3m['f1_score'] = rfc_3m['f1_score'].apply(lambda x: np.nan if x == 0 else x)

In [371]:
rfc_3m.dropna(inplace=True)

In [372]:
rfc_3m.sort_values('precision', ascending=False)

,code,name,precision,f1_score,accuracy,roc_auc
219,009810,['플레이그램'],0.963636,0.762590,0.733871,0.855506
222,010060,['OCI'],0.916667,0.733333,0.935484,0.760482
184,007340,['디티알오토모티브'],0.880000,0.354839,0.354839,0.657374
408,074610,['이엔플러스'],0.848485,0.767123,0.862903,0.932143
420,084010,['대한제강'],0.837209,0.911392,0.943548,0.971749
...,...,...,...,...,...,...
460,123700,['에스제이엠'],0.012821,0.025316,0.379032,0.971545
469,145210,['다이나믹디자인'],0.011905,0.023529,0.330645,0.967480
370,042660,['대우조선해양'],0.011494,0.022727,0.306452,0.898374
455,115390,['락앤락'],0.011364,0.022472,0.298387,0.589431


In [373]:
rfc_3m = rfc_3m.sort_values('precision', ascending=False)

In [374]:
rfc_3m.to_csv('c:/Users/bitcamp/Desktop/final_data/모델정리/종목별_머신러닝/시연용/rfc_3m_15.csv')

In [8]:
pd.read_csv('c:/Users/bitcamp/Desktop/final_data/모델정리/종목별_머신러닝/시연용/rfc_6개월_15.csv', index_col=0)

,code,name,precision,f1_score,accuracy,roc_auc
75,2710,['TCC스틸'],0.978495,0.866667,0.779528,0.827778
230,10770,['평화홀딩스'],0.952381,0.851064,0.944882,0.988385
312,23150,['MH에탄올'],0.950000,0.904762,0.968504,0.904978
277,14990,['인디에프'],0.944444,0.790698,0.929134,0.830784
106,3780,['진양산업'],0.942029,0.726257,0.614173,0.720588
...,...,...,...,...,...,...
372,42700,['한미반도체'],0.023622,0.046154,0.023622,0.690860
190,7690,['국도화학'],0.015748,0.031008,0.015748,0.992000
411,77500,['유니퀘스트'],0.010753,0.021277,0.275591,0.992063
322,25530,['에스제이엠홀딩스'],0.008000,0.015873,0.023622,0.015873


# chatbot+모델

## DB정보 출력

In [ ]:
# 삼성전자

In [467]:
def DB_close(name):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    stock_df = stock_table[stock_table.name == name]
    stock_df['날짜'] = stock_df['날짜'].astype('datetime64')

    close = str(stock_df[stock_df['날짜'] == '2022-03-30']['종가'].values).strip('[]').strip("''")

    print(f'{name}의 종가는 {close}원 입니다')

In [480]:
def DB_start(name):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    stock_df = stock_table[stock_table.name == name]
    stock_df['날짜'] = stock_df['날짜'].astype('datetime64')

    val = str(stock_df[stock_df['날짜'] == '2022-03-30']['시가'].values).strip('[]').strip("''")

    print(f'{name}의 시가는 {val}원 입니다')

In [477]:
def DB_close_yesterday(name):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    stock_df = stock_table[stock_table.name == name]
    stock_df['날짜'] = stock_df['날짜'].astype('datetime64')

    close = str(stock_df[stock_df['날짜'] == '2022-03-29']['종가'].values).strip('[]').strip("''")

    print(f'{name}의 종가는 {close}원 입니다')

In [473]:
def DB_PER(name):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    stock_df = stock_table[stock_table.name == name]
    stock_df['날짜'] = stock_df['날짜'].astype('datetime64')

    val = str(stock_df[stock_df['날짜'] == '2022-03-30']['PER'].values).strip('[]').strip("''")

    print(f'{name}의 PER은 {val}배 입니다')

In [474]:
def DB_benefit(name):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    stock_df = stock_table[stock_table.name == name]
    stock_df['날짜'] = stock_df['날짜'].astype('datetime64')

    val = str(stock_df[stock_df['날짜'] == '2022-03-30']['영업이익'].values).strip('[]').strip("''")

    print(f'{name}의 영업이익은 {val}원 입니다')

In [470]:
DB_close('삼성전자')

삼성전자의 종가는 69900원 입니다


In [475]:
DB_PER('삼성전자')

삼성전자의 PER은 18.2배 입니다


In [476]:
DB_benefit('삼성전자')

삼성전자의 영업이익은 14121409000000.0원 입니다


In [478]:
DB_close_yesterday('삼성전자')

삼성전자의 종가는 70200원 입니다


In [481]:
DB_start('삼성전자')

삼성전자의 시가는 70300원 입니다


## 수익모델 예측

In [499]:
def company_recomend(date):
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'

    conn = pymysql.connect(
        host = end_point,
        user = user_name,
        password = pw,
    #     db = db,
        charset='utf8'

    )
    cursor = conn.cursor()

    sql = 'use Chatbot_DB;'
    cursor.execute(sql)

    sql = 'select * from stock_table;'

    stock_table = pd.read_sql(sql, conn)
    
    
    stock_table['날짜'] = stock_table['날짜'].astype('datetime64')
    stock_df = stock_table[stock_table['날짜'] == date]
    stock_df = stock_df.drop(['BPS', 'PER', 'PBR', 'EPS', 'DIV_', 'DPS', 'quarter','날짜','등락률','name'], axis=1)
    stock_df = stock_df.astype(str).apply(lambda col : col.apply(lambda x: np.nan if x == 'nan' else x))
    stock_df.dropna(inplace=True)

    hoho = defaultdict(list)
    plus_df = pd.DataFrame()
    hoit_df = pd.DataFrame()
    fail_li = []
    code_li = []
    month_li = ['30일','3개월','6개월']
    per_li = ['5per','10per','15per']

    for code in stock_df['code'][:]:
        try:

            df = pd.read_csv(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_재무데이터/머신러닝_train_test/{month}/test/{code}.csv', index_col=0)
            df.drop(['date','등락률','종목명','수익률','quarter','5per','10per','15per'], axis=1, inplace=True)
            df.rename(columns={'법인세차감전 순이익':'법인세차감전_순이익'}, inplace=True)

            ya_df = pd.concat([stock_df[['code']],stock_df[list(df.columns)]], axis=1)
            x_test = ya_df[ya_df.code == code].iloc[0,1:]

            for monmon in month_li[:]:
                for per in per_li[:]:

                        with open(f'c:/Users/bitcamp/Desktop/final_data/머신러닝_모델/{monmon}_{per}/Randomforest/{code}.pkl','rb') as f:
                                rfc = pickle.load(f)

                        pred = rfc.predict([x_test])

                        col = monmon+'_'+per

                        hoho[col].append(pred[0])
            code_li.append(code)

        except:
#             print(code)
            pass

    plus_df = pd.DataFrame({'code':code_li})

    for col,val in hoho.items():
        plus_df[col] = val
    
#     print(plus_df.head())
    
    max_plus_df = plus_df[plus_df['30일_15per']== 1]
    max_plus_df.reset_index(drop=True, inplace=True)

    pre_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/모델정리/종목별_머신러닝/시연용/rfc_30_15.csv', index_col=0)
    pre_df.code = pre_df.code.apply(lambda x:f'{x:06}')

    company_li = []
    for pre in range(len(pre_df)):
        for coco in range(len(max_plus_df)):
            if max_plus_df.iloc[coco, 0] == pre_df.iloc[pre, 0]:
                company_li.append(pre_df.iloc[pre,1])

    return company_li[1].strip('[]').strip("''"), company_li[2].strip('[]').strip("''"), company_li[3].strip('[]').strip("''")

In [500]:
company_recomend('2022-03-30')

('에스엘', '태경케미컬', '흥아해운')

## 추세예측

In [484]:
def companyy_predict(name, date):
    
    end_point = "chatbot-db.c9x08hbiunuu.ap-northeast-2.rds.amazonaws.com"
    port =3306
    user_name = 'root'
    pw ='123123123'
    
    conn = pymysql.connect(
    host = end_point,
    user = user_name,
    password = pw,
#     db = db,
    charset='utf8'
    
    )
    cursor = conn.cursor()
    
    sql = 'use Name_Code_DB;'
    cursor.execute(sql)
    
    sql = 'select * from name_table;'

    name_df = pd.read_sql(sql, conn)
    
    for na in range(len(name_df)):
        if name == name_df.iloc[na,1]:
            code = name_df.iloc[na,0]
    
    autoencoder = load_model(f'c:/Users/bitcamp/Desktop/final_data/autoencoder/Conv_2/auto_conv_{code}.h5')

    company_df = fdr.DataReader(code)
    stock_close = company_df[['Close']]

    stock_close.columns = ['price']
    stock_close['pct_change'] = stock_close.price.pct_change()
    stock_close['log_ret'] = np.log(com_close.price) - np.log(stock_close.price.shift(1))
    stock_close.dropna(inplace=True)

    x_test = stock_close[stock_close.index == date]

    window_length = 10
    scaler = MinMaxScaler()
    x_test_nonscaled = np.array([stock_close['log_ret'].values[i-window_length:i].reshape(-1, 1) for i in tqdm(range(window_length+1,len(stock_close['log_ret'])))])
    x_test = np.array([scaler.fit_transform(stock_close['log_ret'].values[i-window_length:i].reshape(-1, 1)) for i in tqdm(range(window_length+1,len(stock_close['log_ret'])))])

    y_predict = autoencoder.predict(x_test)
    labels = [1,0]

    label = labels[y_predict[0].argmax()]
    confidence = y_predict[0][y_predict[0].argmax()]
    # print('{} {:.2f}%'.format(label, confidence * 100))

    if label == 1:
        print(f'{name}는 상승할 예정입니다.')
    else:
        print(f'{name}는 하락할 예정입니다.')

In [489]:
companyy_predict('삼성전자','2022-03-30')

100%|████████████████████████████████████████████████████████████████████████████| 5988/5988 [00:00<00:00, 8600.59it/s]


삼성전자는 상승할 예정입니다.


## 업종추천

In [303]:
def kospi_kind_recomend():
    kospi_df = pd.read_csv('c:/Users/bitcamp/Desktop/final_data/업종별/월봉_21_8.csv')

    kospi_corr_df = kospi_df.corr()
    kospi_corr_df = kospi_corr_df[['코스피_5']]

    kospi_corr_df.reset_index(drop=False, inplace=True)
    kospi_corr_df.columns = ['업종', 'kospi']

    # kospi_corr_df.sort_values(by=['kospi'], ascending=False)
    kospi_kind_df = kospi_corr_df.sort_values(by=['kospi'])
    kind_recomend = kospi_kind_df.iloc[0,0]
    return kind_recomend.split('_')[0]

In [304]:
kospi_kind_recomend()

'섬유의복'

## 코스피 예측

In [12]:
def kospi_predict(date):
    kospi = fdr.DataReader('KS11')
    kospi_df = kospi[['Close']]

    stock = pd.DataFrame()
    stock['y'] = kospi_df[['Close']]
    stock['ds'] = kospi_df.index

    stock.reset_index(drop=True, inplace=True)

    train = stock[stock.ds < '2021-01']
    test = stock[stock.ds > '2021-01']

    df_prophet = Prophet(changepoint_prior_scale=0.15, daily_seasonality=True)
    df_prophet.fit(train)

    fcast_time = 100
    df_forecast = df_prophet.make_future_dataframe(periods=fcast_time, freq='D')
    # df_forecast.tail(50)
    df_forecast = df_prophet.predict(df_forecast)

    kospi_updow_df = df_forecast[['ds','yhat','yhat_lower','yhat_upper']]
#     kospi_updow_df['ds'] = kospi_updow_df['ds'].astype('datetime64')

    kospi_1 = str(kospi_updow_df[kospi_updow_df['ds'] == date]['yhat_upper'].values)
    kospi_1 = kospi_1.strip('[]')[:4]

    kospi_2 = str(kospi_updow_df[kospi_updow_df['ds'] == date]['yhat_upper'].values)
    kospi_2 = kospi_2.strip('[]')[:4]

    n = int(kospi_2) - int(kospi_1)

    if n > 0:
        print('코스피지수가 상승할 예정입니다.')
    else :
        print('코스피지수가 하락할 예정입니다.')

In [13]:
kospi_predict('2021-03-30')

코스피지수가 하락할 예정입니다.
